# 줄리아를 생각하다(Think julia)

## Chapter 17. 다중 디스패치

* **제너릭 프로그래밍** (generic programming): 여러 가지 자료형에 대응하도록 코드를 작성하는 것을 의미하는 프로그래밍 방법

* **다중 디스패치** (multiple dispatch): 인수의 자료형에 따라 다르게 동작하도록 하는 방법

### 17.1 자료형 선언 (Type Declarations)

* ```::``` **자료형 주석(type annotation)** : 표현식이나 값이 어떤 자료형이어야 하는지를 명시적으로 알려주는 연산자

* ```::``` 연산자는 **할당문의 왼쪽**에 붙을 수 있음(선언의 일부로서 역할)

* ```::``` 연산자는 **함수 정의 헤더**에 붙을 수 있음

* 줄리아에서 자료형이 생략되었을때는 Any 자료형으로 지정 -> 모든 자료형 포괄

In [1]:
(1 + 2)::Float64

LoadError: TypeError: in typeassert, expected Float64, got a value of type Int64

In [2]:
(1 + 2)::Int64

3

In [3]:
function returnfloat()
    x::Float64 = 100
    x
end

x = returnfloat()

println(x)
println(typeof(x))

100.0
Float64


In [4]:
function sinc(x)::Float64
    if x == 0
        return 1
    end
    sin(x)/(x)
end

y = sinc(10)

println(y)
println(typeof(y))

-0.05440211108893698
Float64


### 17.2 메서드(method)

* 자료형 선언(```::```)은 구조체 정의에 있는 필드에 붙을 수 있음

* printtime 함수에 MyTime 객체만 받아들이는 동작, 즉 메서드(method)를 추가하려면 함수 정의에 있는 매개변수 time에 ```::``` 사용

* 메서드는 특정한 시그니처(signature)를 가진 함수 정의 (printtime의 시그니처는 MyTime형인 매개변수가 하나 있다는 것)

In [5]:
using Printf

struct MyTime
    hour::Int64
    minute::Int64
    second::Int64
end

function printtime(time::MyTime)
    @printf("%02d:%02d:%02d", time.hour, time.minute, time.second)
end

printtime (generic function with 1 method)

In [6]:
start = MyTime(9, 45, 0)

printtime(start)

09:45:00

In [7]:
# 만약 인수의 자료형이 올바르지 않을 경우 에러 발생

start2 = MyTime(9.23, 45.4, 0)

printtime(start2)

LoadError: InexactError: Int64(9.23)

* ```::``` 자료형 주석이 없는 첫 번째 메서드 재정의 후 printtime 함수와 비교

In [8]:
function printtime2(time)
    println("I don't know how to print the argument time.")
end

printtime2 (generic function with 1 method)

In [9]:
printtime(150)

LoadError: MethodError: no method matching printtime(::Int64)
[0mClosest candidates are:
[0m  printtime([91m::MyTime[39m) at In[5]:9

In [10]:
printtime2(150)

I don't know how to print the argument time.


**자료형이 명시된 printtime에서는 에러가 발생하지만, 자료형이 명시되지 않은(Any) printtime2 함수에서는 에러가 발생하지 않음**

### 17.3 추가 예시

* increment 함수에 자료형 명시해 재작성

* 인수의 순서가 바뀔 경우, 자료형이 알맞지 않으므로 자료형에 대한 no method matching 에러가 발생함

In [11]:
function timetoint(time)
    minutes = time.hour * 60 + time.minute
    seconds = minutes *60 + time.second
end

function inttotime(seconds)
    (minutes, second) = (seconds ÷ 60 , seconds %60)
    hour, minute = (minutes ÷ 60 , minutes %60)
    MyTime(hour, minute, second)
end

inttotime (generic function with 1 method)

In [12]:
function increment(time::MyTime, seconds::Int64)
    seconds += timetoint(time)
    inttotime(seconds)
end

increment (generic function with 1 method)

In [13]:
start = MyTime(9, 45, 0)
increment(start, 1337)

MyTime(10, 7, 17)

In [14]:
increment(1337, start)

LoadError: MethodError: no method matching increment(::Int64, ::MyTime)

In [15]:
function isafter(t1::MyTime, t2::MyTime)
    (t1.hour, t1.minute, t1.second) > (t2.hour, t2.minute, t2.second)
end

isafter (generic function with 1 method)

In [16]:
isafter(MyTime(10, 45, 0), MyTime(9, 45, 0)) # t1이 t2 이후이냐를 비교함

true

### 17.4 생성자

* 생성자(constructor)는 객체를 생성하기 위해 호출하는 특별한 함수

* 외부 생성자(outer constructor)

* 사본 생성자(copy constructor)

* 내부 생성자(inner constructor)

In [17]:
# 생성자 
# MyTime(hour, minute, second)
# MyTime(hour::Int64, minute::Int64, second::Int64)


# 외부 생성사(사본 생성자)
function MyTime(time::MyTime)
    MyTime(time.hour, time.minute, time.second)
end


# 내부 생성자 : 불변식 강제
struct MyTime 
    hour :: Int64
    minute :: Int64
    second :: Int64
    function MyTime(hour::Int64 = 0, minute::Int64 = 0, second::Int64 = 0)
        @assert(0 ≤ minute < 60, "Minute is not between 0 and 60.")
        @assert(0 ≤ second < 60, "Second is not between 0 and 60.")
        new(hour, minute, second)
    end
end

* 내부 생성자는 항상 자료형이 선언되고 있는 블록 안에서 정의

* 그 자료형의 객체를 만드는 특별한 함수 new에 접근

* 만일내부 생성자가 하나라도 정의되면, 기본 생성자를 사용할 수 없게 되므로, 필요한 내부 생성자를 명시적으로 모두 작성

In [18]:
# 내부 생성자 : 불변식 강제
# case2) new 함수를 인수 없이 호출하는 메소드 
struct MyTime2
    hour :: Int
    minute :: Int
    second :: Int
    function MyTime(hour::Int64 = 0, minute::Int64 = 0, second::Int64 = 0)
        @assert(0 ≤ minute < 60, "Minute is not between 0 and 60.")
        @assert(0 ≤ second < 60, "Second is not between 0 and 60.")
        time = new()
        time.hour = hour
        time.minute = minute
        time.second = second
        time
    end
end

* 이렇게 하면 재귀적 자료구조 생성 가능

    - 여기서 재귀적이라 함은 필드 중 어떤 것이 구조체 자신인 것을 의미



* 이런 방식으로 생성자를 구현한다면, 구조체는 반드시 가변이어야 함 

    - new 함수로 객체를 생성한 후, 객체의 필드를 수정하기 때문임

### 17.5 show 함수

* show 함수는 객체의 문자열 표현을 돌려주는 특별한 함수

* 아래 예시에선 Base.show 함수에 메서드를 추가함

In [19]:
using Printf

function Base.show(io::IO, time::MyTime)
    @printf(io, "%02d:%02d:%02d", time.hour, time.minute, time.second)
end

In [20]:
time = MyTime(9, 45)

09:45:00

* 객체를 그대로 출력하면 줄리아는 자동적으로 show 함수를 불러줌

* 저자는 새로운 복합 자료형을 작성할 때면, 객체 생성을 편하게 할 수 있기 때문에 항상 외부 생성자를 작성하는 것부터 시작함

* 디버깅에 유용한 **show 메서드를 작성함**

### 17.6 연산자 로버로딩 (Operator Overloading)

* 연산자에 메서드 추가

* 연산자 메서드를 정의하면 사용자 정의 자료형에 대한 연산자의 동작을 지정할 수 있음

    * 예를 들어 두 개의 MyTime 인수를 받는 +라는 이름의 메서드를 정의하면, MyTime 객체에 대해 + 연산자를 쓸 수 있음

In [21]:
import Base.+

function +(t1::MyTime, t2::MyTime)
    seconds = timetoint(t1) + timetoint(t2)
    inttotime(seconds)
end

+ (generic function with 207 methods)

In [22]:
start = MyTime(9, 45)

duration = MyTime(1, 35, 0)

start + duration

11:20:00

* import 문은 연산자에 대한 메서드를 추가할 수 있게 + 연산자를 지역 범위에 추가하는 기능 수행

* MyTime 객체에 + 연산자를 적용하면 줄리아는 추가된 메서드를 호출

* REPL에서 그 결과를 출력하기 위해 show 함수를 이용

* 어떤 연산자가 사용자 정의 자료형에 대해 동작하도록 메서드를 추가하는 것 -> 연산자 오버로딩(operator overloading)

### 17.7 다중 디스패치

* **디스패치(dispatch)** 는 함수가 호출될 때 어떤 메서드를 사용해야 하는지를 정하는 메커니즘

* 줄리아는 주어진 인수의 개수와 자료형에 따라 함수의 메서드가 선택되도록 함

* **다중 디스패치(multiple dispatch)** : 함수의 모든 인수를 이용해, 호출된 메서드를 정하는 방법

In [23]:
function +(time::MyTime, seconds::Int64)
    increment(time, seconds)
end

+ (generic function with 208 methods)

In [24]:
start = MyTime(9, 45)

start + 1337

10:07:17

In [25]:
# 덧셈은 교환 법칙이 성립하므로 메서드를 하나 더 생성

function +(seconds::Int64, time::MyTime)
    time + seconds # 위에서 + 연산자에 대한 오버로딩을 해줬기 때문에 사용 가능 
end

+ (generic function with 209 methods)

In [26]:
1337 + start

10:07:17

### 17.8 제너릭 프로그래밍(generic programming)

* 꼭 필요한 경우 다중 디스패치는 매우 유용한 수단이지만, 쓸일이 많지는 않음

* 여러 자료형의 인수에 대해서도 잘 동작하는 함수를 작성하는 것으로 이런 상황을 피할 수 있음

* 여러가지 자료형에 대해서 동작하는 함수를 **다형(polymorphic)** 함수라고 함

* **다형성(polymorphism)** 은 코드 재사용을 용이하게 함

In [27]:
# MyTime 객체에 대해 + 메서드를 만들었으므로 sum 함수를 사용할 수 있음

t1 = MyTime(1, 7, 2)
t2 = MyTime(1, 5, 8)
t3 = MyTime(1, 5, 0)

sum((t1, t2, t3))

03:17:10

* 일반적으로 함수 내부에 있는 모든 연산이 주어진 자료형에 대해 잘 동작한다면, 그 함수는 그 자료형에 대해 잘 동작한다고 말함

* 다형성 중에서 가장 좋은 종류는 의도하지 않은 다형성(의외의 자료형에 대해 함수가 잘 동작하는 상황 등)

### 17.9 인터페이스와 구현

* 다중 디스패치의 목표는 소프트웨어에 대한 쉬운 유지 보수

* 이를 위해서는 인터페이스와 구현 분리 필요 -> (자료형이 주어진 인수를 가진 메서드는 그 자료형의 필드가 어떻게 표현되는지에 의존하지 않아야 함)

* 인터페이스를 디자인할 때 주의를 기울인다면, 인터페이스의 변경 없이 구현만 바꿀 수 있음

### 17.10 디버깅

* 함수의 메서드를 확인하는 함수 **method**

In [29]:
function printtime(time)
    h = time.hour
    m = time.minute
    s = time.second
    
    println("$h:$m:$s")
end


methods(printtime)

# 2 methods for generic function "printtime":
[1] printtime(time::MyTime) in Main at In[5]:9
[2] printtime(time) in Main at In[29]:1

```
```
**->** 함수 printtime의 메서드는 두 개로 나타남 

* 하나는 MyTime을 인수로 갖는 것

* 다른 하나는 Any 타입을 인수로 갖는 것

### 17.11 용어집

* **자료형 주석** (type annotation)

    - 자료형이 뒤에 붙는 ```::``` 연산자는 어떤 표현식이나 변수가 그 자료형임을 나타냄
```
```
* **메서드** (method)

    - 어떤 함수에 대해 가능한 동작을 정의한 것.
```
```
* **시그니처** (signature)

    - 메서드의 인수 개수와 그것들의 자료형. 디스패치는 이 시그니처를 이용해 호출된 함수에 대해 선택 가능한 메서드들 중에서 인수가 가장 구체적인 것을 선택함
```
```
* **생성자** (constructor)

    - 객체를 생성하기 위해 호출하는 특별 함수
```
```
* **기본 생성자** (default constructor)

    - 개발자가 정의한 내부 생성자가 없을 경우에 사용 가능한 내부 생성자
```
```
* **외부 생성자** (outer constructor)

    - 자료형의 정의 바깥에서 정의되는 생성자로 객체 생성을 편하게 하기 위한 메서드
```
```
* **사본 생성자** (copy constructor)

    - 어떤 자료형의 외부 생성자로서 그 자료형의 객체가 유일한 인수인 것.인수의 복사본을 생성함
```
```
* **내부 생성자** (inner constructor)

    - 자료형의 정의 내부에서 정의되는 생성자로 불변식을 강제하거나 자가 참조 객체를 만들기 위해 사용
```
```
* **연산자 오버로딩** (operator overloading)

    - ```+``` 같은 연산자가 사용자 정의 자료형에 대해서도 잘 동작하도록 메서드를 추가하는 것
```
```
* **디스패치** (dispatch)

    - 함수가 호출되었을 때, 어떤 메서드를 실행해야 할지 선택하는 것
```
```
* **다중 디스패치** (multiple dispatch)

    - 함수의 모든 인수에 기반해 디스패치하는 것
```
```
* **제너릭 프로그래밍** (generic programming)

    - 두 종류 이상의 자료형에 대해 잘 동작하는 코드를 작성하는 것
```
```
* **다형 함수** (polymorphic function)

    - 인수가 여러 자료형일 수 있는 함수
